In [1]:
import torch
from trainUtils import print_classification_report, get_classification_report, load_dataset
from models.modeling import VisionTransformer, CONFIGS

device = torch.device("cuda:{}".format(0)) if torch.cuda.is_available() else torch.device("cpu")

In [17]:
# Build dataloaders
train_loader, test_loader, _ = load_dataset('../serif_dataset/images/train',
                                       '../serif_dataset/images/test',
                                       train_batch_size=32,
                                       resize_size=600,
                                       crop_size=448,
                                       num_workers=8)
config = CONFIGS['ViT-B_16']
config.split = 'overlap'
config.slide_step = 12

/share/softwares/anaconda/anaconda3/envs/svt/lib/python3.8/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [7]:
model = VisionTransformer(config, 448,
                        zero_head=True,
                        num_classes=4,
                        smoothing_value=0.0,
                        class_weights_path=None)
ckpt = torch.load('/share/data/drive_2/repos/GEST_serifs/TransFG/output/b16_checkpoint.bin')
model.load_state_dict(ckpt['model'], strict=True)
model.to(device);

In [8]:
report_dict_train = get_classification_report(model, train_loader, device)
report_train = print_classification_report(report_dict_train)
print(report_train)

100%|██████████| 92/92 [03:11<00:00,  2.08s/it]


In [9]:
report_dict_test = get_classification_report(model, test_loader, device)
report_test = print_classification_report(report_dict_test)
print(report_test)

              precision    recall  f1-score   support

         0.0       0.94      0.39      0.55       620
         1.0       0.92      0.97      0.94      1178
         2.0       0.83      0.63      0.72       558
         3.0       0.54      0.97      0.70       558

    accuracy                           0.78      2914
   macro avg       0.81      0.74      0.73      2914
weighted avg       0.83      0.78      0.77      2914



In [12]:
p = report_dict['weighted avg']['precision']
r = report_dict['weighted avg']['recall']
f = report_dict['weighted avg']['f1-score']

f1 = 2*((p*r)/(p+r))

print(f, f1)

0.7680147568365381 0.8063389668506007


In [16]:
((0.55*620 + 0.94*1178 + 0.72*558 + 0.70*558)/2914)

0.768936170212766